In [1]:
import os
import numpy as np
import pydicom

import pandas as pd
import sys
sys.path.append('/mnt/fast-disk1/mjc/utils_codes/read_weasis_raw_v0.96/')

import weasis_raw_data_api as wr
sys.path.append('/mnt/fast-disk1/mjc/utils_codes/')
from utils_test import *
from utils_metrics_3d import *

D_dir2header_df = {}
def get_dicom_header_df(image_dir , labels = []):
    global D_dir2header_df
    if image_dir in D_dir2header_df:
        return D_dir2header_df[image_dir]

    # image_dir = row['Image File Path']


    labels = ['ImageName','InstanceNumber',
            'BitsAllocated', 'BitsStored', 'Columns', 'HighBit', 
            'ImageOrientationPatient_0', 'ImageOrientationPatient_1', 'ImageOrientationPatient_2',
            'ImageOrientationPatient_3', 'ImageOrientationPatient_4', 'ImageOrientationPatient_5',
            'ImagePositionPatient_0', 'ImagePositionPatient_1', 'ImagePositionPatient_2',
            'Modality', 'PatientID', 'PhotometricInterpretation', 'PixelRepresentation',
            'PixelSpacing_0', 'PixelSpacing_1', 'RescaleIntercept', 'RescaleSlope', 'Rows', 'SOPInstanceUID',
            'SamplesPerPixel', 'SeriesInstanceUID', 'StudyID', 'StudyInstanceUID', 
            'WindowCenter', 'WindowWidth', 
        ] if not labels else labels

    data = {l: [] for l in labels}
    
    ctList = os.listdir(image_dir)
    ctList.sort()

    for image in ctList:
        if '.dcm' not in image:
            continue
        if os.path.getsize(os.path.join(image_dir, image)) < 5*1024:
            print('%s size < 5kb skiped!'%os.path.join(image_dir, image) )
            continue
        data["ImageName"].append(image)

        ds = pydicom.dcmread(os.path.join(image_dir, image))
        for metadata in ds.dir():
            if metadata not in data and metadata not in ['ImageOrientationPatient','ImagePositionPatient','PixelSpacing']:
                continue
            if metadata != "PixelData":
                metadata_values = getattr(ds, metadata)
                if type(metadata_values) == pydicom.multival.MultiValue and metadata not in ["WindowCenter", "WindowWidth"]:
                    for i, v in enumerate(metadata_values):
                        data[f"{metadata}_{i}"].append(v)  
                else:

                    if type(metadata_values) == pydicom.multival.MultiValue and metadata in ["WindowCenter", "WindowWidth"]:
                        data[metadata].append(metadata_values[0])
                    else:
                        
                        if metadata in ['ImageOrientationPatient','ImagePositionPatient','PixelSpacing']:
                            print( 'error of loading key: {}'.format(metadata) )                    
                        else:
                            data[metadata].append(metadata_values)

    df_image = pd.DataFrame(data).set_index("InstanceNumber")
    D_dir2header_df[image_dir] = df_image
    return df_image

In [2]:
def pd_str_replace(df , col, ori, new):
    if isinstance(col , str):
        try:
            df[col] = df[col].str.replace(ori,new, case = False) 
        except:
            pass
            
    elif isinstance(col, list):
        for one in col:
            pd_str_replace(df , one, ori, new)
    else:
        raise('col instance should be str or list')


def str_Xdrive2mnt(df_all):
    pd_str_replace(df_all, ['Image File Path' , 'Contour File Path'], "X:" , "/mnt/Y-drive")
    pd_str_replace(df_all, ['Image File Path' , 'Contour File Path'], r"\\" , "/")
    pd_str_replace(df_all, ['Image File Path'], "/mnt/Y-drive/ClinicalTrials/FNIH_VOLPACK", "/mnt/fast-disk1/mjc/AutoRecist/Inputs")
    pd_str_replace(df_all, ['Image File Path'], "/mnt/Y-drive/ClinicalTrialDone/FNIH_VOLPACK", "/mnt/fast-disk1/mjc/AutoRecist/Inputs")
    pd_str_replace(df_all, ['Image File Path'], "/mnt/Y-drive/ClinicalTrials", "/mnt/fast-disk1/mjc/AutoRecist/Inputs")

    pd_str_replace(df_all, ['Contour File Path'], "/mnt/Y-drive/ConvWeasisToRaw/PDS_AUTO_RECIST_Modified_By_Yen",
    "/mnt/fast-disk1/mjc/AutoRecist/Inputs/ConvWeasisToRaw/PDS_AUTO_RECIST_Modified_By_Yen")
    pd_str_replace(df_all, ['Contour File Path'], "/mnt/Y-drive/ConvWeasisToRaw/PDS_AUTO_RECIST", "/mnt/fast-disk1/mjc/AutoRecist/Inputs/ConvWeasisToRaw/PDS_AUTO_RECIST_RAW")
    pd_str_replace(df_all, ['Contour File Path'], "/mnt/Y-drive/ConvWeasisToRaw", "/mnt/fast-disk1/mjc/AutoRecist/Inputs/ConvWeasisToRaw")
    pd_str_replace(df_all, ['Contour File Path'], "/mnt/Y-drive/ConvWeasisToMatlab", "/mnt/fast-disk1/mjc/AutoRecist/Inputs/ConvWeasisToRaw")
    
def get_onect_from_list(df_list , ct):
    for i, df in enumerate(df_list):
        try:
            df_ct = df[ (df["Image File Path"]==ct) & (df['Location'].isin(['liver'])) ]
        except KeyError:
            df_ct = df[ (df["Image File Path"]==ct)]
        if df_ct.shape[0]:
            return df_ct , i
    print("warning! no CT was found")
    return None ,None

def raws2mask(raws , D_z_index, mask_vol = None):

    for raw in raws:

        radiologist_raw = wr.read(raw)
        slice_list = radiologist_raw.get_instance_number_array()
        if mask_vol is None:
            mask_vol = initialize_mask_vol(radiologist_raw , D_z_index)
        for j, one in enumerate(slice_list):
            mask = radiologist_raw.get_mask_image(j)
            mask_vol[D_z_index[one]] += mask
    return mask_vol

In [3]:
folder = '/mnt/fast-data/mjc/AutoRECIST/Inputs/'



df_CTs = pd.read_excel(folder+'AutoRECIST_List_LesionSize_20220602_JM_SingleCTSeries.xlsx')
str_Xdrive2mnt(df_CTs)


df_Yen = pd.read_excel(folder+'PDS_AUTO_RECIST CIA-LAB Testing Dataset Gold Standard_Yen_2022-06-13.xlsx')
str_Xdrive2mnt(df_Yen)

AI_raw_list = ['ScaleNAS9SliceToRaw_Test353.csv',]
df_AIs = []
for one in AI_raw_list:
    df = pd.read_csv(one)
    str_Xdrive2mnt(df)
    df_AIs.append(df)




Metrics_vol = []
metrics_save_path = 'Metrics_%s_vs_Yen353pts.csv'%'ScaleNAS9Slices'
CTs = df_CTs["Image File Path"].values.tolist()
import time
for ct in CTs:
    t = time.time()
    df_image = get_dicom_header_df( ct )
    instanceNumber_list = df_image.index.to_list()
    D_z_index = instanceNumber2Matrix_z_index(instanceNumber_list)


    df_ct_Yen = df_Yen[df_Yen["Image File Path"]==ct]
    df_ct_AI , dataset_id = get_onect_from_list(df_AIs , ct)
    # break
    if (df_ct_AI is None):
        if df_ct_Yen.shape[0]:
            fn = df_ct_Yen.shape[0]
            print('{} has {} FNs!'.format(ct , fn))   
        continue

    if not df_ct_Yen.shape[0]:
        fp = df_ct_AI.shape[0]
        print( '{} has {} FPs'.format(ct , fp)  )
        continue
    else:
        print(ct)

    raws = df_ct_AI["Contour File Path"].values.tolist()
    
    elapsed = time.time() - t
    print('raws time cost is {} sec'.format(elapsed) )
    
    vol_pred = raws2mask(raws , D_z_index, mask_vol = None)
    elapsed = time.time() - t
    print('vol_pred time cost is {} sec'.format(elapsed) )
    connectivity = 2
    from skimage import measure
    labels_pred=measure.label(vol_pred,connectivity=connectivity)
    l_pred,c_pred = np.unique(labels_pred , return_counts=True)
    ix2 = l_pred>0
    l_pred = l_pred[ix2] #background pixels are labeled as 0, so we exclude them
    c_pred = c_pred[ix2]
    if len(l_pred)!= len(raws):
        print("warning! raws overlaped on {}".format(ct))
    elapsed = time.time() - t
    print('c_pred time cost is {} sec'.format(elapsed) )

    for _ , row in df_ct_Yen.iterrows():
        t = time.time()
        Yen_raw = wr.read(row['Contour File Path'])
        gt_vol = initialize_mask_vol(Yen_raw , D_z_index)
        

        slice_list = Yen_raw.get_instance_number_array()
        for j, one in enumerate(slice_list):
            mask = Yen_raw.get_mask_image(j)
            gt_vol[D_z_index[one]] = mask
        elapsed = time.time() - t
        print('gt_vol time cost is {} sec'.format(elapsed) )
        hit = vols_seg_results(gt_vol , vol_pred, CTname=row['Contour File Path'], gt_keep_largest=1)
        Metrics_vol.extend(hit)
        elapsed = time.time() - t
        print('vols_seg_results time cost is {} sec'.format(elapsed) )

#         _n = len(Metrics_vol)
#         if _n%100==0 or _n in [1,2,5,10,30,50]:
#             df_metrics = pd.DataFrame(Metrics_vol, 
#                                     columns = ['file_name','igt','merge','#gt','#pred',
#                                                 'iou_score', 'dice_score', 'over_seg' , 'under_seg',
#                                                 'area_gt','area_pred','intersection','union']) 
#             df_metrics.to_csv(metrics_save_path)
        


/mnt/fast-disk1/mjc/AutoRecist/Inputs/PDS_AUTO_RECIST/METNET0002/D2019_01_02/E1326/CT/S0007_3337
raws time cost is 0.3735804557800293 sec
vol_pred time cost is 0.3861560821533203 sec
warning! raws overlaped on /mnt/fast-disk1/mjc/AutoRecist/Inputs/PDS_AUTO_RECIST/METNET0002/D2019_01_02/E1326/CT/S0007_3337
c_pred time cost is 1.8347127437591553 sec
gt_vol time cost is 0.0034494400024414062 sec
vols_seg_results time cost is 7.1484456062316895 sec
gt_vol time cost is 0.0024390220642089844 sec
vols_seg_results time cost is 7.035311222076416 sec
gt_vol time cost is 0.005133867263793945 sec
vols_seg_results time cost is 7.253568649291992 sec
gt_vol time cost is 0.002465963363647461 sec
vols_seg_results time cost is 7.195111513137817 sec
gt_vol time cost is 0.005255460739135742 sec
vols_seg_results time cost is 7.104291200637817 sec
gt_vol time cost is 0.0031404495239257812 sec
vols_seg_results time cost is 7.166154384613037 sec
/mnt/fast-disk1/mjc/AutoRecist/Inputs/PDS_AUTO_RECIST/METNET0003

KeyboardInterrupt: 

In [ ]:
df_metrics = pd.DataFrame(Metrics_vol, 
                        columns = ['file_name','igt','merge','#gt','#pred',
                                    'iou_score', 'dice_score', 'over_seg' , 'under_seg',
                                    'area_gt','area_pred','intersection','union']) 
df_metrics.to_csv(metrics_save_path)

In [ ]:
pd.options.display.float_format = "{:.3f}".format
df_metrics = pd.DataFrame(Metrics_vol, 
                          columns = ['file_name','igt','merge','#gt','#pred',
                                     'iou_score', 'dice_score', 'over_seg' , 'under_seg',
                                     'area_gt','area_pred','intersection','union']) 
df_metrics.describe([.05, .25, .5, .75, .95])

In [ ]:
df_metrics[df_metrics.dice_score>0.25].describe([.05, .25, .5, .75, .95])

In [ ]:
dfinnermerge = pd.merge(df_metrics,df_Yen,how='inner',left_on='file_name' , right_on='Contour File Path')
for col in dfinnermerge.columns.tolist():
    print(col , len(set(dfinnermerge[col].tolist() )) )



pts = dfinnermerge["Image File Path"].values.tolist()
FPs = []
for onept in list(set(pts)):
    df_onept = dfinnermerge[dfinnermerge["Image File Path"]==onept]
    assert( min(df_onept["#pred"]) == max(df_onept["#pred"]))
    fp = max(df_onept["#pred"])
    FPs.append(fp)
print("="*80)
print( "In total, {} CT series; {} AI detections ".format( len(FPs) , sum(FPs) ) )

dices = dfinnermerge.dice_score.tolist()

for th in [0, 0.1, 0.2, 0.25, 0.5]:
    TP = [p>th for p in dices ]
    assert( len(TP) == len(dices))
    fprate = ( sum(FPs) - sum(TP) ) / len(FPs)
    print(f"sensitivity is {sum(TP)/len(dices):.3f}({sum(TP)}/{len(dices)}) FP-rate is {fprate:.1f} per CT-serie at threshold {th}")

In [ ]:
df_metrics